# Intro to Flipside API/SDK: Getting Started

<em>install Flipside with pip</em><br/>
`pip install flipside`

Import the package

In [ ]:
from flipside import Flipside

Run your first query<br/>
<em>Remember to copy/paste your API Key from https://flipsidecrypto.xyz/account/api-keys below.</em>

In [ ]:
import os
YOUR_API_KEY = os.environ.get("FLIPSIDE_API_KEY")

# Invoke the ShroomDK class to create an instance of the SDK
sdk = Flipside(YOUR_API_KEY)

# Run a query
xMETRIC_contract_address = '0x15848C9672e99be386807b9101f83A16EB017bb5'

query_result_set = sdk.query(f"""
    SELECT count(distinct to_address) as recipient_count
    FROM polygon.core.fact_token_transfers
    WHERE block_timestamp > '2022-07-10T00:00:00'
        AND contract_address = lower('{xMETRIC_contract_address}')
        AND to_address != lower('0x4b8923746a1D9943bbd408F477572762801efE4d')
""")


### Query Result Set

```python
class QueryResultSet(BaseModel):
    query_id: Union[str, None] = Field(None, description="The server id of the query")
    status: str = Field(False, description="The status of the query (`PENDING`, `FINISHED`, `ERROR`)")
    columns: Union[List[str], None] = Field(None, description="The names of the columns in the result set")
    column_types: Union[List[str], None] = Field(None, description="The type of the columns in the result set")
    rows: Union[List[Any], None] = Field(None, description="The results of the query")
    run_stats: Union[QueryRunStats, None] = Field(
        None,
        description="Summary stats on the query run (i.e. the number of rows returned, the elapsed time, etc)",
    )
    records: Union[List[Any], None] = Field(None, description="The results of the query transformed as an array of objects")
    error: Any

```

In [ ]:
## Explore the result set object

records = query_result_set.records

print(records[0])

print(f"There are {records[0]['recipient_count']} unique recipients of xMETRIC tokens.")

### xMETRIC Leaderboard
Retrieve the balance of every xMETRIC holder

In [ ]:
WITH sent_tokens AS (
  SELECT 
    to_address as Participant,
    sum(raw_amount/pow(10,18)) as xMETRIC
  FROM polygon.core.fact_token_transfers
  WHERE
    block_timestamp::date > '2022-07-10'::date 
    AND contract_address = lower('0x15848C9672e99be386807b9101f83A16EB017bb5')
    AND to_address != lower('0x4b8923746a1D9943bbd408F477572762801efE4d')
  GROUP BY 1
),
burnt_tokens AS (
  SELECT
    to_address as Participant,
    sum(raw_amount/pow(10,18)) as xMETRIC
  FROM polygon.core.fact_token_transfers
  WHERE
    block_timestamp::date > '2022-07-10'::date 
    AND contract_address = lower('0x15848C9672e99be386807b9101f83A16EB017bb5')
    AND to_address = lower('0x0000000000000000000000000000000000000000')
  GROUP BY 1
)
SELECT
  sent_tokens.Participant as "participant_addr",
  coalesce(sent_tokens.xmetric,0) - coalesce(burnt_tokens.xMETRIC,0) as "balance"
FROM sent_tokens 
LEFT JOIN burnt_tokens ON sent_tokens.Participant = burnt_tokens.Participant
ORDER BY 2 DESC

In [ ]:
# Load the sql query from a file
leaderboard_sql_query = open("./sql/xmetric_leaderboard.sql", 'r').read()

# Run the query with pagination

page_number = 1
page_size = 10

leaderboard_result_set = sdk.query(
    leaderboard_sql_query, 
    page_size=page_size,
    page_number=page_number)

for record in leaderboard_result_set.records:
    print(record)

### Plot the xMETRIC LeaderBoard Results

In [ ]:
full_leaderboard_result_set = sdk.query(leaderboard_sql_query)

import pandas as pd
import plotly.express as px

df = pd.DataFrame(full_leaderboard_result_set.records)

fig = px.histogram(df, x="balance", marginal="box", hover_data=df.columns, nbins=200)

fig.show()

### Cross Chain xMETRIC User Exploration

In [ ]:
WITH xmetric_holders AS (
    SELECT to_address as holder_addr
    FROM polygon.core.fact_token_transfers
    WHERE block_timestamp > '2022-07-10T00:00:00'
        AND contract_address = lower('0x15848C9672e99be386807b9101f83A16EB017bb5')
        AND to_address != lower('0x4b8923746a1D9943bbd408F477572762801efE4d')
)
SELECT
    token_name,
    symbol,
    count(distinct user_address) as num_holders,
    median(usd_value_now) as median_usd_holdings
FROM ethereum.core.ez_current_balances
INNER JOIN xmetric_holders 
    ON ethereum.core.ez_current_balances.user_address = xmetric_holders.holder_addr
WHERE ethereum.core.ez_current_balances.usd_value_now > 0
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 25

In [ ]:
# Load the sql query from a file
xmetric_eth_holdings_sql_query = open("./sql/xmetric_eth_holdings.sql", 'r').read()

# Run the query
xmetric_eth_holdings_results = sdk.query(xmetric_eth_holdings_sql_query)


In [ ]:
# Plot the results
df = pd.DataFrame(xmetric_eth_holdings_results.records)

fig = px.bar(df, x="token_name", y="num_holders", hover_data=df.columns)

fig.show()